In [0]:
import pandas as pd

silver_path = "/Workspace/social media/Silver/youtube_silver_cleaned.csv"
gold_folder = "/Workspace/social media/Gold"

df_silver = pd.read_csv(silver_path)

print("Silver Data Loaded:")
df_silver.head()


Silver Data Loaded:


,comment_id,comment_text,likes,replies,username,user_channel,date,url,video_id,replies_value,username_md5,replies_without_names,comment_length
0,UgwHDXadD9Hhyzbq6bp4AaABAg,Oh boy another Hell lady Transmog….,3,1,@ed███96,https://www.youtube.com/@eddy796,NaN,https://www.youtube.com/watch?v=dHDsP_XvFH0&pp...,dHDsP_XvFH0,NaN,74ad40a3854b3b33d49d0188c1fa1af4,"[{""reply_id"":""UgwHDXadD9Hhyzbq6bp4AaABAg.ABcyP...",35
1,Ugy7bU1m3foDO3wQNaJ4AaABAg,new transmogs again already? theyre gettin des...,1,1,@ra███mco███nto██████,https://www.youtube.com/@randomcommentor5474,NaN,https://www.youtube.com/watch?v=dHDsP_XvFH0&pp...,dHDsP_XvFH0,NaN,9913c91d38c02fdd8fd2c1454df8dbc3,"[{""reply_id"":""Ugy7bU1m3foDO3wQNaJ4AaABAg.ABd8V...",62
2,UgwhzVRKOj1z8ak-LEF4AaABAg,I NEED the hell lady transmog. They are my fav...,5,2,@ko███rst███r77███,https://www.youtube.com/@konnorstever7776,NaN,https://www.youtube.com/watch?v=dHDsP_XvFH0&pp...,dHDsP_XvFH0,NaN,89e0a313769d90569aa1a753fde31b02,"[{""reply_id"":""UgwhzVRKOj1z8ak-LEF4AaABAg.ABczq...",101
3,UgyMfS3lJ8vpFQhgPAB4AaABAg,Primeiro meu rei,2,1,@Ma███sed███do7███,https://www.youtube.com/@Marcuseduardo7,NaN,https://www.youtube.com/watch?v=FpDkEHWttaQ&pp...,FpDkEHWttaQ,NaN,237fe738a05aa2cf0a6331c2ecb8cbf9,"[{""reply_id"":""UgyMfS3lJ8vpFQhgPAB4AaABAg.ABhyR...",16
4,UgyRJS3P13IlyRUptfV4AaABAg,Horseradish sauce FTW. Thanks for the recipe.,3,6,@er███wrd███,https://www.youtube.com/@ercedwrds,NaN,https://www.youtube.com/watch?v=1O8MMKerNMs&pp...,1O8MMKerNMs,NaN,6d8a981c2df8790b18cf21d95c12a7e3,"[{""reply_id"":""UgyRJS3P13IlyRUptfV4AaABAg.ABiMA...",45


In [0]:
import numpy as np

df_gold = df_silver.copy()

# Engagement Score
df_gold["engagement_score"] = df_gold["likes"].fillna(0) + df_gold["replies"].fillna(0)

# User-level aggregates
user_stats = df_gold.groupby("username").agg({
    "comment_id": "count",
    "likes": "sum",
    "replies": "sum",
    "engagement_score": "sum"
}).reset_index()

user_stats.rename(columns={"comment_id": "total_comments"}, inplace=True)

user_stats.head()


,username,total_comments,likes,replies,engagement_score
0,@10███wor███,1,29,6,35
1,@19███izz███,1,4,2,6
2,@1a███tir███,1,2,4,6
3,@1l███bab███,1,2,4,6
4,@20███bas███,1,1,0,1


In [0]:
gold_path = f"{gold_folder}/youtube_gold_aggregated.csv"

user_stats.to_csv(gold_path, index=False)

print("Gold File Saved Successfully At:", gold_path)



Gold File Saved Successfully At: /Workspace/social media/Gold/youtube_gold_aggregated.csv


In [0]:
# DIM USER TABLE
dim_user = df_gold[["username", "user_channel"]].drop_duplicates()

dim_user["user_id"] = dim_user["username"].factorize()[0] + 1

dim_user_path = f"{gold_folder}/dim_user.csv"
dim_user.to_csv(dim_user_path, index=False)

print("Created:", dim_user_path)


Created: /Workspace/social media/Gold/dim_user.csv


In [0]:
dim_video = df_gold[["video_id", "url"]].drop_duplicates()

dim_video["video_sk"] = dim_video["video_id"].factorize()[0] + 1

dim_video_path = f"{gold_folder}/dim_video.csv"
dim_video.to_csv(dim_video_path, index=False)

print("Created:", dim_video_path)


Created: /Workspace/social media/Gold/dim_video.csv


In [0]:
df_gold["date"] = pd.to_datetime(df_gold["date"], errors="coerce")

dim_date = pd.DataFrame()
dim_date["date"] = df_gold["date"].dropna().unique()

dim_date["date"] = pd.to_datetime(dim_date["date"])
dim_date["date_sk"] = dim_date["date"].dt.strftime("%Y%m%d").astype(int)
dim_date["year"] = dim_date["date"].dt.year
dim_date["month"] = dim_date["date"].dt.month
dim_date["day"] = dim_date["date"].dt.day
dim_date["day_of_week"] = dim_date["date"].dt.day_name()

dim_date_path = f"{gold_folder}/dim_date.csv"
dim_date.to_csv(dim_date_path, index=False)

print("Created:", dim_date_path)


Created: /Workspace/social media/Gold/dim_date.csv


In [0]:
fact_comments = df_gold[[
    "comment_id", "comment_text", "likes", "replies",
    "username", "video_id", "date"
]].copy()

# Join surrogate keys
fact_comments = fact_comments.merge(dim_user, on="username", how="left")
fact_comments = fact_comments.merge(dim_video, on="video_id", how="left")
fact_comments = fact_comments.merge(dim_date, on="date", how="left")

fact_comments_path = f"{gold_folder}/fact_comments.csv"
fact_comments.to_csv(fact_comments_path, index=False)

print("Created:", fact_comments_path)

# If date is missing, generate from year, month, day
if "date" not in fact_comments.columns or fact_comments["date"].isna().all():

    print("⚠ Date is missing — generating from year, month, day...")

    fact_comments["date"] = pd.to_datetime(
        fact_comments[["year", "month", "day"]],
        errors="coerce"
    )

    # If any rows fail to create a date, fill with today's date
    fact_comments["date"] = fact_comments["date"].fillna(pd.Timestamp.today())

print("Generated Date Column:")
print(fact_comments[["year", "month", "day", "date"]].head())



Created: /Workspace/social media/Gold/fact_comments.csv
⚠ Date is missing — generating from year, month, day...
Generated Date Column:
   year  month  day                       date
0   NaN    NaN  NaN 2025-11-18 06:28:54.151588
1   NaN    NaN  NaN 2025-11-18 06:28:54.151588
2   NaN    NaN  NaN 2025-11-18 06:28:54.151588
3   NaN    NaN  NaN 2025-11-18 06:28:54.151588
4   NaN    NaN  NaN 2025-11-18 06:28:54.151588


In [0]:
fact_user_stats_path = f"{gold_folder}/fact_user_stats.csv"
user_stats.to_csv(fact_user_stats_path, index=False)

print("Created:", fact_user_stats_path)


Created: /Workspace/social media/Gold/fact_user_stats.csv
